## Trabajando con RNNS
### Comenzamos con una red neuronal recurrente simple

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

2025-08-19 20:29:07.857608: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-19 20:29:07.868409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755646147.880815  444193 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755646147.884698  444193 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755646147.894739  444193 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
# con esto le ensenho al modelo
x = np.array([[[0], [1], [2]], [[1], [2], [3]]]) # secuencia

# esto es lo que quiero que mi modelo trate de predecir
y = np.array([3, 4])

# defino mi modelo de rnn simple
model = Sequential([
  SimpleRNN(10, activation="tanh", input_shape=(3, 1)),
  Dense(1)
])

model.compile(optimizer="adam", loss="MSE")
model.fit(x, y, epochs=200, verbose=0)

print("Prediccion del modelo para [2, 3, 4]: ", model.predict(np.array([[[2], [3], [4]]])))

/home/frejala/Code/deep-learning-course/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Prediccion del modelo para [2, 3, 4]:  [[2.8636582]]


Caso de uso para prediccion de temperaturas

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler

In [5]:
# traemos el dataset
df = pd.read_csv(r"temperature.csv")

temps = df["San Francisco"].dropna().values.reshape(-1, 1)

# Normalizacion
scaler = MinMaxScaler()
temps_scaled = scaler.fit_transform(temps)

x, y = [], []

for i in range(len(temps_scaled) - 25):
	x.append(temps_scaled[i:i+24])
	y.append(temps_scaled[i+24])

x, y = np.array(x), np.array(y)

model = Sequential([
	SimpleRNN(50, activation="tanh", input_shape=(24, 1)),
	Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(x, y, epochs=10, batch_size=64)

pred = model.predict(x[:1])
print("Prediccion para la proxima hora: ", scaler.inverse_transform(pred))

Epoch 1/10


/home/frejala/Code/deep-learning-course/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0016
Epoch 2/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.8635e-04
Epoch 3/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.3297e-04
Epoch 4/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.2717e-04
Epoch 5/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.2383e-04
Epoch 6/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.1889e-04
Epoch 7/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.1935e-04
Epoch 8/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.1994e-04
Epoch 9/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.2265e-04
Epoch 10/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.1716e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Prediccion para la proxima hora:  [[289.39523]]


In [7]:
pred = model(x[:1])
pred_kelvin = scaler.inverse_transform(pred)[0][0]
pred_celsius = pred_kelvin - 273.15
print(f"Prediccion de la siguiente hora: {pred_kelvin:.2f} K / {pred_celsius:.2f}ºC")

Prediccion de la siguiente hora: 289.40 K / 16.25ºC


RNN Bidireccional

In [8]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences


# cargamos nuestra base de datos
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

# Padding de secuencias
maxlen = 200
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

model = Sequential([
	Embedding(10000, 128, input_length=maxlen),
	Bidirectional(LSTM(64)),
	Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

# evaluamos nuestro modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5


/home/frejala/Code/deep-learning-course/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 87ms/step - accuracy: 0.8039 - loss: 0.4135 - val_accuracy: 0.8522 - val_loss: 0.3639
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 97ms/step - accuracy: 0.9063 - loss: 0.2365 - val_accuracy: 0.8710 - val_loss: 0.3191
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - accuracy: 0.9329 - loss: 0.1772 - val_accuracy: 0.8587 - val_loss: 0.3508
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 98ms/step - accuracy: 0.9566 - loss: 0.1206 - val_accuracy: 0.8570 - val_loss: 0.3773
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 104ms/step - accuracy: 0.9668 - loss: 0.0962 - val_accuracy: 0.8585 - val_loss: 0.4752
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.8585 - loss: 0.4752
Loss: 0.4752, Accuracy: 0.8585
